In [16]:
import numpy as np # Numerical computing library
import matplotlib.pyplot as plt # Plotting library
import scipy.integrate #Integration library
from mpl_toolkits.mplot3d import axes3d #Used for the 3d bifurcation plot
import matplotlib.patches as mpatches #used to write custom legends
from scipy.special import binom
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import sympy as sym
import math as m


from matplotlib import animation, rc
#import matplotlib.animation as animation
#%matplotlib notebook
#%matplotlib inline
from IPython.display import HTML

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

import scipy.integrate as integrate
from sympy.abc import x, y, S
from multiprocessing import Pool
import os
from datetime import datetime

%matplotlib inline

Useful functions

In [17]:
def gauss(s):
    return(1/m.sqrt(m.pi)*m.exp(-s**2))

def cut_compact(x):
    if abs(x)<=0.5:
        return(3./4-x**2)
    else:
        if 0.5<abs(x)<=1.5:
            return(0.5*(1.5-abs(x))**2)
        else:
            return(0)

Advection function

In [18]:
def pos(x,L): #identifie the index of the posisition of x in the ordonned list L

    if x>=L[-1]:
        return(len(L))
    else:
        i=0
        while x>=L[i]:
            i=i+1
        return(i)
    
def affine_by_part(L_root, L_slope, m, M):              # return the function with roots L_root, with slope L_slope at each of these roots 
                                                        # and bounded by (m,M)
    L_change=[(L_slope[i]*L_root[i]-L_slope[i+1]*L_root[i+1])/(L_slope[i]-L_slope[i+1]) for i in range(len(L_root)-1)]
    def func_(x):
        return(L_slope[pos(x, L_change)]*(x-L_root[pos(x, L_change)]))
            
    def func(x):
        return(min(M, max(func_(x), m)))
    return(func)



def affine_by_part_der(L_root, L_slope, m, M):              # return the function with roots L_root, with slope L_slope at each of these roots 
                                                        # and bounded by (m,M)
    L_change=[(L_slope[i]*L_root[i]-L_slope[i+1]*L_root[i+1])/(L_slope[i]-L_slope[i+1]) for i in range(len(L_root)-1)]
    def func(x):
        return(L_slope[pos(x, L_change)])
    return(func)




I_stable_left=np.array([150000.0, 208817.63527054107])
I_stable_center=np.array([193286.5731462926, 224649.2985971944])
I_stable_right=np.array([185270.54108216433, 250000.0])

I_unstable_short=np.array([193286.5731462926, 208817.63527054107])
I_unstable_long=np.array([185270.54108216433, 224649.2985971944])

x_stable_left_coef=np.array([-1.98068372e-20,  1.72778767e-14, -6.01655669e-09,  1.04558220e-03,
 -9.08181301e+01,  3.18506176e+06])
x_stable_center_coef=np.array([-9.97960480e-19,  1.04447036e-12, -4.37203191e-07,  9.14932022e-02,
 -9.57246289e+03,  4.00596473e+08])
x_stable_right_coef=np.array([-6.10906482e-21,  6.84633959e-15, -3.06591965e-09,  6.85909353e-04,
 -7.66847769e+01,  3.43040294e+06])
x_unstable_short_coef=np.array([ 1.98171027e-17, -1.99617838e-11,  8.04269750e-06, -1.62015993e+00,
  1.63180639e+05, -6.57391347e+09])
x_unstable_long_coef=np.array([ 2.50429596e-19, -2.55194809e-13,  1.03983466e-07, -2.11771508e-02,
  2.15571288e+03, -8.77460219e+07])

def x_stable_left(S):
    return(np.dot(x_stable_left_coef, np.array([S**5, S**4, S**3, S**2, S, 1])))
def x_stable_center(S):
    return(np.dot(x_stable_center_coef, np.array([S**5, S**4, S**3, S**2, S, 1])))
def x_stable_right(S):
    return(np.dot(x_stable_right_coef, np.array([S**5, S**4, S**3, S**2, S, 1])))
def x_unstable_short(S):
    return(np.dot(x_unstable_short_coef, np.array([S**5, S**4, S**3, S**2, S, 1])))
def x_unstable_long(S):
    return(np.dot(x_unstable_long_coef, np.array([S**5, S**4, S**3, S**2, S, 1])))

def f_multi(x,S):
    #C1, C2, C3, C4, C5=p[0], p[1], p[2], p[3], p[4]
    
    C=0.02
    
    C1=C
    C2=C
    C3=C
    C4=C
    C5=C
    M=10**10
    m=-10**10
    
    if S<=I_stable_right[0]:
        return(min(M, max(-C1*(x-x_stable_left(S)), m)))
        
    if I_stable_right[0]<S<=I_stable_center[0]:
        L_slope=[-C5,C4, -C1]
        L_root=[x_stable_right(S), x_unstable_long(S), x_stable_left(S)]
        #print(L_root)
        return(affine_by_part(L_root, L_slope, m, M)(x))
    
    if I_stable_center[0]<S<=I_stable_left[-1]:
        L_slope=[-C5, C4, -C3, C2, -C1]
        L_root=[ x_stable_right(S),x_unstable_long(S), x_stable_center(S), x_unstable_short(S), x_stable_left(S)]
        return(affine_by_part(L_root, L_slope, m, M)(x))
    
    if I_stable_left[-1]<S<=I_stable_center[-1]:
        L_slope=[-C5,C4, -C3]
        L_root=[x_stable_right(S), x_unstable_long(S), x_stable_center(S)]
        return(affine_by_part(L_root, L_slope, m, M)(x))
    
    if S>I_stable_center[-1]:
        return(min(M, max(-C5*(x-x_stable_right(S)), m)))

def f_prime_multi(x,S):

    #C1, C2, C3, C4, C5=p[0], p[1], p[2], p[3], p[4]
     
    C=0.02
    
    C1=C
    C2=C
    C3=C
    C4=C
    C5=C
    M=10**10
    m=-10**10
    
    if S<=I_stable_right[0]:
        return(-C1)
        
    if I_stable_right[0]<S<=I_stable_center[0]:
        L_slope=[-C5,C4, -C1]
        L_root=[x_stable_right(S), x_unstable_long(S), x_stable_left(S)]
        #print(L_root)
        return(affine_by_part_der(L_root, L_slope, m, M)(x))
    
    if I_stable_center[0]<S<=I_stable_left[-1]:
        L_slope=[-C5, C4, -C3, C2, -C1]
        L_root=[ x_stable_right(S),x_unstable_long(S), x_stable_center(S), x_unstable_short(S), x_stable_left(S)]
        return(affine_by_part_der(L_root, L_slope, m, M)(x))
    
    if I_stable_left[-1]<S<=I_stable_center[-1]:
        L_slope=[-C5,C4, -C3]
        L_root=[x_stable_right(S), x_unstable_long(S), x_stable_center(S)]
        return(affine_by_part_der(L_root, L_slope, m, M)(x))
    
    if S>I_stable_center[-1]:
        return(-C5)

    
    
def f(x):
    global S_cons
    return(f_multi(x, S_cons))


def f_prime(x):
    global S_cons
    return(f_prime_multi(x,S_cons))




In [19]:
#Compute the solution for an initial condition in [0,1]^2

def approx(functions, x0, Lt, cut_off, eps_x): 
    a, a_prime, r, khi, M, n0= functions[0], functions[1], functions[2], functions[3], functions[4], functions[5]
    
    Nx=len(x0)
    h=(x0[-1]-x0[0])/Nx
    x0_=np.array([y for y in x0])
    delta=Lt[0]
    
    w0=np.array([h for i in range(Nx)])      
    v0=np.array([h*n0(x0_[i]) for i in  range(Nx)])
    X0=np.concatenate((x0_, w0, v0))
    
    L_fin_1=[]
    L_fin_2=[]

    def F(t,Y):
        X=Y[0:Nx]
        W=Y[Nx:2*Nx]
        V=Y[2*Nx:3*Nx]
        F_1=np.array([a(X[i]) for i in range(Nx)])
        F_2=np.array([a_prime(X[i]) for i in range(Nx)])*W
        aux1=np.array([[M(X[i], X[j])*V[j] for j in range(Nx)] for i in range(Nx)])
        aux2=np.array([[M(X[j], X[i])*W[j] for j in range(Nx)] for i in range(Nx)])
        aux3=np.array([[khi(X[i], X[j])*V[j] for j in range(Nx)] for i in range(Nx)])
        F_3=np.array([(r(X[i])-np.sum(aux3[i])-np.sum(aux2[i]))*V[i]+W[i]*np.sum(aux1[i]) for i in range(Nx)])
        return(np.concatenate((F_1, F_2, F_3)))
    
    
    def ker(s):
        return(1./(eps_x)*cut_off(s/eps_x))
    
    for i in range(len(Lt)):
        sol=solve_ivp(F, (0, delta), X0, t_eval=[delta]).y
        sol_x=sol[0:Nx]
        sol_v=sol[2*Nx:3*Nx]
        
        SUM=np.array([np.sum(np.array([sol_v[k][0]*ker(x0_[i]-sol_x[k][0])  for k in range(Nx)])) for i in range(Nx)])
   #     SUM_=np.array([np.sum(SUM[i*NS:(i+1)*NS]) for i in range(Nx)])*h_S
        L_fin_1.append(SUM)
  #      L_fin_2.append(SUM_)
        v0=SUM*w0

        X0=np.concatenate((x0_, w0, v0))
        
        if i<len(Lt)-1:
            delta=Lt[i+1]-Lt[i]
            
    return(np.array(L_fin_1))

#Compute the solution for an initial condition in any domain
def approx_re(functions, x0, Lt, cut_off, eps_x): 
    a, a_prime, r, d, M, n0= functions[0], functions[1], functions[2], functions[3], functions[4], functions[5]
    
    A=x0[-1]-x0[0]
    B=x0[0]

    
    def a_re(x):
        return((1./A)*a(A*x+B))
    
    def a_prime_re(x):
        return(a_prime(A*x+B))
    
    def r_re(x):
        return(r(A*x+B))
    
#    def d_re(x, S): 
#        return(d(A*x+B, C*S+D))
    def khi_re(x, y):
        return(khi(A*x+B, A*y+B))
    
    def M_re(x, y):
        return(A*M(A*x+B, A*y+B))
  
    
    def n0_re(x):
        return(A*n0(A*x+B))
    
    functions_re=[a_re, a_prime_re, r_re, khi_re, M_re, n0_re]
    
    Nx=len(x0)
    
    x0_re=np.linspace(0, 1, Nx)
    
    U=approx(functions_re, x0_re, Lt, cut_off, eps_x)
    return(1./(A)*U)

Changeable parameters

In [20]:
Nx=20

x0=np.linspace(0*1000, 25*1000, Nx)

hx=(x0[-1]-x0[0])/(Nx)


r0=0.0182
#Growth term

def r(x):
    return(0.0182)

def r_heterogeneous(x,p, q): 
    if x>=16*1000:
        return(r0)
    
    if 4*1000<x<16*1000:
        return(r0/p)
        
    if x<=4*1000:
        return(r0/q)

carrying_capacity=10*1000



#eta=5*1000
#eta=10*1000
#eta=20*1000
#eta=30*1000
#eta=40*1000

#tau=1.

#def M(x,S,y,S_):
    
#    return(1./tau*1./eta*1./(x0[-1]-x0[0])*gauss((S-S_)/eta)*r(x,S))



           

#tau_ep, tau_mes=1, tau_hyb=2, 5, 10
           
def tau_heterogeneous(x, tau_ep, tau_hyb, tau_mes):
           
    if x>=16*1000:
        return(tau_ep)
    
    if 4*1000<x<16*1000:
        return(tau_hyb)
        
    if x<=4*1000:
        return(tau_mes)

    


coeff_poly_tau=np.polyfit([1*1000, 12.5*1000, 25*1000], [2,1,5],2)
def tau_polynomial(x):
#P(x=1)=2
#P(x=12.5)=1
#P(x=25)=5
    return(np.dot([x**2, x, 1], coeff_poly_tau))

#plt.plot(x0, [tau_polynomial(x) for x in x0])
#plt.show()

#Initial population size
init_size=100

#Initial epithelial population

def n0_ep(x):
    if  16*1000<=x<=25*1000:
        return(init_size/(25*1000-16*1000))
    else:
        return(0)

#Initial hybrid population

def n0_hyb(x):
    
    if   4*1000<=x<=16*1000:
        return(init_size/(16*1000-4*1000))
    else:
        return(0)
    
#Initial mesenchymal population

def n0_mes(x):
    if  0<=x<=4*1000:
        return(init_size/(4*1000))
    
    else:
        return(0)

def n0_ep_mes(x):
    return(1./2*(n0_ep(x)+n0_mes(x)))

def n0_ep_hyb(x):
    return(1./2*(n0_ep(x)+n0_hyb(x)))

def n0_hyb_mes(x):
    return(1./2*(n0_hyb(x)+n0_mes(x)))

def n0_ep_hyb_mes(x):
    return(1./3*(n0_ep(x)+n0_hyb(x)+n0_mes(x)))

def n0_uni(x):
    return(init_size/(25*1000))
    
#functions=[f, f_prime, r, d, M, n0_ep] 

#Times where the solution is computed 
Lt=[5*24, 10*24, 15*24, 20*24, 25*24, 30*24, 35*24, 40*24, 45*24, 50*24, 55*24, 60*24, 65*24, 70*24, 75*24, 80*24, 85*24, 90*24, 95*24, 100*24, 110*24, 120*24, 130*24, 140*24, 150*24, 160*24, 170*24, 180*24, 190*24, 200*24, 300*24, 400*24, 500*24 ] 
#Lt=[1*24, 2*24] 

In [22]:
carrying_capacity=10*1000
S_cons=210*1000
eta_x= 2000
tau = 1

r_het_p = 1
r_het_q = 1

alpha_m_m=1
alpha_h_m=1
alpha_e_m=1
alpha_m_h=1
alpha_h_h=1
alpha_e_h=1
alpha_m_e=1
alpha_h_e=1
alpha_e_e=1

def M(x,y):
    return(1./tau*1./(eta_x)*gauss((x-y)/eta_x)*r(x))

def r_heterogeneous(x,p, q): 
    if x>=16*1000:
        return(r0)
    
    if 4*1000<x<16*1000:
        return(r0/p)
        
    if x<=4*1000:
        return(r0/q)
    
def r(x):
    return(r_heterogeneous(x,r_het_p,r_het_q))    

def ind_M(y):

    if y<=4*1000:
        return(1)
    else:
        return(0)

def ind_H(y):
    
    if  4*1000<y<=16*1000:
        return(1)
    else:
        return(0)
    
def ind_E(y):
    if 16*1000<y<=25*1000:
        return(1)
    else:
        return(0)

def khi(y, z):

    d_m= r(y)*1./carrying_capacity
    d_h=r(y)*1./carrying_capacity
    d_e=r(y)*1./carrying_capacity

    if y<=4*1000 :
        return(d_m*(alpha_m_m *ind_M(z)+alpha_h_m*ind_H(z)+alpha_e_m*ind_E(z)))

    if  4*1000<y<=16*1000:
        return(d_h*(alpha_m_h *ind_M(z)+alpha_h_h*ind_H(z)+alpha_e_h*ind_E(z)))
        
    if 16*1000<y<=25*1000: 
        return(d_e*(alpha_m_e *ind_M(z)+alpha_h_e*ind_H(z)+alpha_e_e*ind_E(z)))

   

Compute the solution

In [23]:
eps_x=(1./Nx)**0.8

In [24]:


def para_set_parallel_sim(para_set):
    
    global r_het_p, r_het_q, alpha_e_h, alpha_e_m, alpha_h_e, alpha_h_m, alpha_m_e, alpha_m_h

    r_het_p = para_set[0]
    r_het_q = para_set[1]
    init_pop_indx = int(para_set[2])
    alpha_h_m=para_set[3]
    alpha_e_m=para_set[3]
    alpha_m_h=para_set[4]
    alpha_m_e=para_set[4]
    alpha_e_h=1
    alpha_h_e=1
    
    init_pop = [n0_ep, n0_hyb, n0_mes, n0_ep_mes,n0_ep_hyb,n0_hyb_mes,n0_uni,n0_ep_hyb_mes]    
    functions=[f, f_prime, r, khi, M, init_pop[init_pop_indx]] 
    U=approx_re(functions, x0, Lt, gauss, eps_x)    
    file_name = ['Sol_s_lim_', str('%.2f' % S_cons), '_eta_x_', str('%.2f' % eta_x), '_r_het_p_', str('%.2f' % para_set[0]), '_r_het_q_', str('%.2f' % para_set[1]), '_ini_pop_', str('%.2f' % para_set[2]), '_alpha_eh_m_', str('%.2f' % para_set[3]), '_alpha_m_eh_', str('%.2f' % para_set[4]),  '.txt' ] 
    file_name = ''.join(file_name) 
    op=open(file_name, 'w+')
    np.savetxt(file_name, U)    
    return(U)

In [28]:
# initialing all parameters

os.chdir('/mnt/c/Users/user/OneDrive - Indian Institute of Science/Projects/EMT PDE population model - Jules/sim_data_updated_model')

# complete range of parameters to be sampled

r_het_p_q_list = [[1, 1]] #, [1, 2]] #, [2, 2]]
alpha_eh_m = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]
alpha_m_eh = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]


i = 0
para_set = np.zeros([len(alpha_m_eh)*len(alpha_eh_m)*len(r_het_p_q_list)*6,5])

for r_het_p_q_indx in range(len(r_het_p_q_list)):
    for pop_indx in [0, 1, 2, 3, 6, 7]:
            for alpha_eh_m_indx in range(len(alpha_eh_m)):
                             for alpha_m_eh_indx in range(len(alpha_m_eh)):
                                para_set[i] = [r_het_p_q_list[r_het_p_q_indx][0], r_het_p_q_list[r_het_p_q_indx][1], pop_indx, alpha_eh_m[alpha_eh_m_indx], alpha_m_eh[alpha_m_eh_indx]]
                                i = i+1


file_name = ['Para_sets for r_het_p r_het_q ini_pop alpha_eh_m alpha_m_eh.txt'] 
file_name = ''.join(file_name)
op=open(file_name, 'w+')
np.savetxt(file_name,para_set)


In [29]:
os.chdir('/mnt/c/Users/user/OneDrive - Indian Institute of Science/Projects/EMT PDE population model - Jules/sim_data_updated_model')

Solution = []
file_name = ['Para_sets for r_het_p r_het_q ini_pop alpha_eh_m alpha_m_eh.txt'] 
file_name = ''.join(file_name)
para_set = np.loadtxt(file_name)
filter_indx = []

#os.chdir('/mnt/c/Users/user/OneDrive - Indian Institute of Science/Projects/EMT PDE population model - Jules/sim_data')

for indx in range(len(para_set)):
    file_name = ['Sol_s_lim_', str('%.2f' % S_cons), '_eta_x_', str('%.2f' % eta_x), 'r_het_p_', str('%.2f' % para_set[indx,0]), '_r_het_q_', str('%.2f' % para_set[indx,1]), '_ini_pop_', str('%.2f' % para_set[indx,2]), '_alpha_eh_m_', str('%.2f' % para_set[indx,3]), '_alpha_m_eh_', str('%.2f' % para_set[indx,4]),  '.txt' ] 
    file_name = ''.join(file_name)
    if(os.path.isfile(file_name) == True):
        filter_indx.append(indx)
        US_read = np.loadtxt(file_name)
        file_name_update = ['Sol_s_lim_', str('%.2f' % S_cons), '_eta_x_', str('%.2f' % eta_x), '_r_het_p_', str('%.2f' % para_set[indx,0]), '_r_het_q_', str('%.2f' % para_set[indx,1]), '_ini_pop_', str('%.2f' % para_set[indx,2]), '_alpha_eh_m_', str('%.2f' % para_set[indx,3]), '_alpha_m_eh_', str('%.2f' % para_set[indx,4]),  '.txt' ] 
        file_name_update = ''.join(file_name_update) 
        op=open(file_name_update, 'w+')
        np.savetxt(file_name_update, US_read)
        os.remove(file_name)


In [30]:

Solution = []
file_name = ['Para_sets for r_het_p r_het_q ini_pop alpha_eh_m alpha_m_eh.txt'] 
file_name = ''.join(file_name)
para_set = np.loadtxt(file_name)
filter_indx = []

os.chdir('/mnt/c/Users/user/OneDrive - Indian Institute of Science/Projects/EMT PDE population model - Jules/sim_data_updated_model')

for indx in range(len(para_set)):
    file_name = ['Sol_s_lim_', str('%.2f' % S_cons), '_eta_x_', str('%.2f' % eta_x), '_r_het_p_', str('%.2f' % para_set[indx,0]), '_r_het_q_', str('%.2f' % para_set[indx,1]), '_ini_pop_', str('%.2f' % para_set[indx,2]), '_alpha_eh_m_', str('%.2f' % para_set[indx,3]), '_alpha_m_eh_', str('%.2f' % para_set[indx,4]),  '.txt' ] 
    file_name = ''.join(file_name)
    if(os.path.isfile(file_name) == False):
        filter_indx.append(indx)
    
    else:
        sol = np.loadtxt(file_name)
        if(len(sol)!= len(Lt)):  
            os.remove(file_name)
            filter_indx.append(indx)
para_set = para_set[filter_indx]
        

In [31]:
# Running parallel simultaneously
num_cores = 1
start=datetime.now()
if __name__ == '__main__':
    with Pool(num_cores) as pool:
        for result in pool.map(para_set_parallel_sim, para_set):
            Solution.append(result)
time=datetime.now()-start